In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

In [22]:
df_mod = pd.read_csv('Terry_Cleaned.csv')
df_mod = df_mod.drop(['Unnamed: 0','Officer Squad'], axis=1)
df_mod.head()

,Subject Age Group,Stop Resolution,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Weapon,Call Made,Arrest Made,Frisk,Year Reported
0,26 - 35,Arrest,1984,M,Black or African American,Asian,Male,0,0,0,0,2015
1,26 - 35,Field Contact,1965,M,White,Other,Male,0,0,0,0,2015
2,26 - 35,Field Contact,1961,M,White,White,Male,0,0,0,0,2015
3,26 - 35,Field Contact,1963,M,White,Other,Male,0,0,0,0,2015
4,26 - 35,Field Contact,1977,M,White,Black or African American,Male,0,0,0,0,2015


In [24]:
# Check for class imbalance before split
print(df_mod['Arrest Made'].value_counts())

0    40890
1     2173
Name: Arrest Made, dtype: int64


In [25]:
# Create X and y
y = df_mod['Arrest Made']
X = df_mod.drop(['Arrest Made'], axis=1)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Remove object-type features from X
cont_features = X.select_dtypes(exclude = ['object'])

# Remove object-type features X_train and X_test
X_train_cont = X_train.select_dtypes(exclude = ['object'])
X_test_cont = X_test.select_dtypes(exclude = ['object'])


In [26]:
# Scale the train and test data
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train_cont)
X_test_ss = ss.transform(X_test_cont)

In [27]:
# Create a dataframe only containing the object-type features

X_train_cat = X_train.select_dtypes(include = ['object'])
X_test_cat = X_test.select_dtypes(include = ['object'])

In [29]:
# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Convert these columns into a DataFrame
columns = ohe.get_feature_names(input_features=X_train_cat.columns) # Build an array of column names
cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=columns) # Return a dense version of the dataframe
cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=columns)

In [82]:
col = cont_features.columns

In [83]:
dff = pd.DataFrame(X_train_ss, columns=col)
dff.head()

,Officer YOB,Weapon,Call Made,Frisk,Year Reported
0,0.275346,-0.569282,0.653443,1.878759,1.054549
1,-0.065742,-0.569282,-1.530356,-0.532266,-0.854468
2,0.957524,1.756600,0.653443,-0.532266,1.054549
3,-0.179438,-0.569282,0.653443,1.878759,1.054549
4,-1.430097,-0.569282,-1.530356,-0.532266,-1.490807


In [84]:
# Combine scaled training data and categorcial data

X_train_all = pd.concat([pd.DataFrame(X_train_ss, columns=col), cat_train_df], axis=1)
X_test_all = pd.concat([pd.DataFrame(X_test_ss, columns=col), cat_test_df], axis=1)

In [85]:
X_test_all.head()

,Officer YOB,Weapon,Call Made,Frisk,Year Reported,Subject Age Group_1 - 17,Subject Age Group_18 - 25,Subject Age Group_26 - 35,Subject Age Group_36 - 45,Subject Age Group_46 - 55,...,Subject Perceived Race_Black or African American,Subject Perceived Race_Hispanic,Subject Perceived Race_Multi-Racial,Subject Perceived Race_Native Hawaiian or Other Pacific Islander,Subject Perceived Race_Other,Subject Perceived Race_Unknown,Subject Perceived Race_White,Subject Perceived Gender_Female,Subject Perceived Gender_Gender Diverse (gender non-conforming and/or transgender),Subject Perceived Gender_Male
0,-1.316401,-0.569282,-1.530356,-0.532266,-1.490807,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.957524,-0.569282,0.653443,-0.532266,-0.218129,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.957524,1.756600,0.653443,-0.532266,1.690888,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.047954,-0.569282,0.653443,-0.532266,1.054549,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.065742,-0.569282,0.653443,-0.532266,-0.854468,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [86]:
# Check for class imbalance
print(y_train.value_counts())

# Fit SMOTE to training data
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(X_train_all, y_train)

# Preview synthtic sample class distrubution
print('\n')
print(pd.Series(y_train_resampled).value_counts())

0    30634
1     1663
Name: Arrest Made, dtype: int64


1    30634
0    30634
dtype: int64


In [87]:
classifiers = [KNeighborsClassifier(), 
               RandomForestClassifier(), AdaBoostClassifier(), 
              GradientBoostingClassifier()]

for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train_resampled, y_train_resampled)
    print(classifier)
    print('model score: %.3f' %pipe.score(X_test_all, y_test))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
model score: 0.989
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model score: 0.997
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)
model score: 0.997
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_

### Need help choosing and formatting paramters (see below)

In [99]:
pipe

Pipeline(memory=None,
         steps=[('classifier',
                 GradientBoostingClassifier(criterion='friedman_mse', init=None,
                                            learning_rate=0.1, loss='deviance',
                                            max_depth=3, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100,
                                            n_iter_no_change=None,
                                            presort='auto', random_state=None,
                                            subsample=1.0, tol=0.0001,
                       

In [100]:
grid = {"classifier__n_estimators": [5, 10, 50, 100],
        "classifier__min_samples_leaf": [0.10, 1]}

In [101]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'classifier', 'classifier__criterion', 'classifier__init', 'classifier__learning_rate', 'classifier__loss', 'classifier__max_depth', 'classifier__max_features', 'classifier__max_leaf_nodes', 'classifier__min_impurity_decrease', 'classifier__min_impurity_split', 'classifier__min_samples_leaf', 'classifier__min_samples_split', 'classifier__min_weight_fraction_leaf', 'classifier__n_estimators', 'classifier__n_iter_no_change', 'classifier__presort', 'classifier__random_state', 'classifier__subsample', 'classifier__tol', 'classifier__validation_fraction', 'classifier__verbose', 'classifier__warm_start'])

In [104]:
gridsearch = GridSearchCV(estimator=pipe, param_grid=grid, 
                          scoring='accuracy', cv=5, verbose=1, n_jobs)



In [105]:
gridsearch.fit(X_train_resampled, y_train_resampled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [90]:
clf = KNeighborsClassifier()
clf.fit(X_train_resampled, y_train_resampled)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [91]:
clf.score(X_test_all, y_test)

0.9894111090469998

In [95]:
# from sklearn.feature_selection import RFE, RFECV
# rfe = RFECV(clf, cv=5)
# X_rfe_train = rfe.fit_transform(X_train_resampled, y_train_resampled)
# X_rfe_test = rfe.transform(X_test_all)

In [77]:
# dicts= [
#              {
#                "name": "KNeighborsClassifier",
#                "estimator": KNeighborsClassifier(),
#                "hyperparameters": 
#                  {
#                    "n_neighbors": range(1,20,2),
#                    "weights": ["distance", "uniform"],
#                    "algorithm": ["ball_tree", "kd_tree", "brute"],
#                    "p": [1,2]
#                  },
            
#              {
#                "name": "GradientBoostingClassifier",
#                "estimator": GradientBoostingClassifier(),
#                "hyperparameters": 
#                  {
#                    "n_estimators": [5, 20, 100],  
#                    "max_features": ["auto", "log2", "sqrt"],
#                    "learning_rate":[0.01, 0.05, 0.1, 0.5],
#                    "subsample":[0.1, 0.5, 1.0],  
#                    "random_state":[1]     
#                  }
#              ,
#              {
#                "name": "AdaBoostClassifier",
#                "estimator": AdaBoostClassifier(),
#                "hyperparameters": 
#                  {
#                    "n_estimators": [5, 20, 100],                     
#                    "learning_rate":[0.01, 0.05, 0.1, 0.5],
#                    "algorithm": ['SAMME','SAMME.R'],  
#                    "random_state":[1]     
#                  }
#              ,
#              {
#                "name": "RandomForestClassifier",
#                "estimator": RandomForestClassifier(),
#                "hyperparameters": 
#                  {     
#                    "n_estimators": [5, 20, 100],  
#                    "criterion": ["entropy", "gini"],
#                    "max_depth": [2, 5, 10],
#                    "max_features": ["log2", "sqrt"],
#                    "min_samples_leaf": [1, 5, 8],
#                    "min_samples_split": [2, 3, 5], 
#                    "class_weight": [None, "balanced", {0: 1, 1: 3}, {0: 1, 1: 5}]               
#                  }]    